# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [433]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
import pylab as plt

import warnings
warnings.filterwarnings('ignore')

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0
    #random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3
    #random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
7000


## Пишем свой класс для SVM

In [434]:
import numpy as np
from random import randint
import random


#np.random.seed(42)
#random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
        #return 1/(1 + np.exp(-self.f(x))) #sigmoid
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return 1.0 * sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.a(x)])
        #return - answer * np.log(self.a(x)) #cross-entropy

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -answer if answer * self.a(x) < 1 else 0
        #return -answer * (1 - self.a(x)) #cross-entropy

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        loss = []
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** np.log(k + 2)

            # w update
            self.w = self.w - step * (self.der_loss(x, y) * x) - self.der_reg()
            
            # w_0 update
            self.w0 = self.w0 - step * self.der_loss(x, y)
            
            if k % 10 == 0:
                loss.append(self.loss(x, y))
            
        return loss

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [436]:
model = MySVM()
loss = model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.84792281  0.96779698 -2.24583096  0.0409238  -2.17115445 -0.66552221
 -0.32945185  1.68899902  2.42781375  1.84748184  0.76841355 -1.93818977
  1.91508398  0.35570549 -1.70773312  1.37367207 -0.46609084 -3.64265787
 -2.34073907  2.99480892] 6.78927895814501


In [437]:
predictions = model.predict(X_test)

In [438]:
print(predictions)

[0 1 0 ... 1 1 1]


In [439]:
print(y_test, len(y_test), sum(y_test))

[0 1 0 ... 1 1 0] 3000 1533


In [440]:
print(len(predictions), sum(predictions))

3000 2434


In [441]:
print(accuracy_score(y_test, predictions.round()))

0.6523333333333333


In [442]:
accuracy = []
C = 1
while C <= 10000:
    model = MySVM(C)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, predictions.round()))
    C *= 10

In [443]:
np.round(accuracy, decimals=4)

array([0.489 , 0.6007, 0.634 , 0.6077, 0.6357])

## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [444]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(C = 10000, verbose=True)

In [445]:
SVC.fit(X_train, y_train)

[LibLinear]

LinearSVC(C=10000, verbose=True)

In [446]:
SVC.coef_

array([[-0.21306179, -0.03287832, -0.1267871 , -0.19987849, -0.08661255,
         0.11155433,  0.0773211 ,  0.07427582,  0.08466198,  0.17262518,
         0.00549195, -0.10685271, -0.01086645, -0.01503025, -0.06344994,
        -0.15477311, -0.09046428, -0.0868942 ,  0.05677665,  0.07998461]])

In [447]:
pred = SVC.predict(X_test)

In [448]:
accuracy_score(y_test, pred)

0.8136666666666666

In [449]:
C = 1
SVC_accuracy = []
while C <= 10000:
    SVC = LinearSVC(C=C, max_iter = 10000)
    SVC.fit(X_train, y_train)
    pred = SVC.predict(X_test)
    SVC_accuracy.append(accuracy_score(y_test, pred))
    C *= 10

In [450]:
np.round(SVC_accuracy, decimals=4)

array([0.8787, 0.8793, 0.837 , 0.8243, 0.783 ])